1) Load the necessary libraries and define any specialized functions

In [1]:
import datetime, json, os, time, uuid, random
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import janitor
from janitor import clean_names, remove_empty

from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankableAction, RewardRequest, RankRequest
from msrest.authentication import CognitiveServicesCredentials

def remove_common(a, b):
    common = set(a) & set(b)
    a = [i for i in a if i not in common]
    b = [i for i in b if i not in common]
    return(a)

def save_xls(dict_df, path):
    """
    Save a dictionary of dataframes to an excel file, 
    with each dataframe as a separate page
    """

    writer = ExcelWriter(path)
    for key in dict_df.keys():
        dict_df[key].to_excel(writer, sheet_name=key)

    writer.close()

2) Output from the web portal will come in here as `.xlsx` and `.csv`s, which will be loaded into `pandas` dataframes and worked on by `pyjanitor`'s `clean_names`

In [ ]:
xls = pd.ExcelFile('/PHShome/tpd10/Gits/bwh-pharmacoepi-roybal/inputs/Sample_CSV_REINFORCE-EHR_2024-12-10.xlsx')

xls.sheet_names

variables_to_exclude = {"Med Generic Name", "Study ID", "PAT_STUDY_ID", "Pat Study ID", "Pat_Study_ID", "Order Creator Study ID", "Authorizing Provider Study ID", 'Discontinue Provider Study ID', "Provider_Study_ID", "FU_CURRENT_ICD10_LIST", "ICD10_CODE"}

3) `pyjanitor` calls `clean_names` are run here to reduce all cases of all *srings* and *object* variable types to lower case, and replace *ALL* spaces and *SOME* special characters with underscores. The exception is for medication doses expressed in decimals (e.g. 0.5 mg), which is written in as an exception.

The call itself is documented here:

https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.clean_names.clean_names

With the key arguments being:
- axis = None, to let `pyjanitor` know you don't want it to work on the column names (just the values)
- column_names = varlist, which is a `list` of columns upon which you want the cleaning to be performed

In [2]:

sheet_to_df_map = {}
for name in range(len(xls.sheet_names)):
    sheet_to_df_map[xls.sheet_names[name]] = xls.parse(xls.sheet_names[name])
    varlist = sheet_to_df_map[xls.sheet_names[name]].dtypes[sheet_to_df_map[xls.sheet_names[name]].dtypes != 'int64'][sheet_to_df_map[xls.sheet_names[name]].dtypes != 'float64'][sheet_to_df_map[xls.sheet_names[name]].dtypes !='datetime64[ns]'].index.tolist()
    varlist = remove_common(varlist,variables_to_exclude)
    if len(varlist) > 0:
        sheet_to_df_map[xls.sheet_names[name]] = sheet_to_df_map[xls.sheet_names[name]].clean_names(axis=None, column_names=varlist, remove_special=False)
        if any("Med Generic Name" in s for s in sheet_to_df_map[xls.sheet_names[name]]):
            sheet_to_df_map[xls.sheet_names[name]]["Med Generic Name"] = sheet_to_df_map[xls.sheet_names[name]]["Med Generic Name"].str.lower().str.replace(' ','_')
    else:
        continue
    
    
save_xls(dict_df = sheet_to_df_map, path = '/PHShome/tpd10/Gits/bwh-pharmacoepi-roybal/inputs/cleaned_example_Sample_CSV_REINFORCE-EHR_2024-12-10.xlsx')


3) API/Azure connection workflow for **submitting** data to Azure Personalizer is executed below

4) API/Azure connection workflow for **retrieving** features that has been extracted by Azure Personalizer is executed below